In [66]:
import pandas as pd
from sklearn.impute import SimpleImputer
import numpy as np
from numpy import nan
import math
from numpy import mean
from numpy import std
from pandas import read_csv
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline


from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

In [67]:
df=pd.read_csv("~\Desktop\movieReplicationSet.csv")

In [68]:
df.head()

,The Life of David Gale (2003),Wing Commander (1999),Django Unchained (2012),Alien (1979),Indiana Jones and the Last Crusade (1989),Snatch (2000),Rambo: First Blood Part II (1985),Fargo (1996),Let the Right One In (2008),Black Swan (2010),...,When watching a movie I cheer or shout or talk or curse at the screen,When watching a movie I feel like the things on the screen are happening to me,As a movie unfolds I start to have problems keeping track of events that happened earlier,"The emotions on the screen ""rub off"" on me - for instance if something sad is happening I get sad or if something frightening is happening I get scared",When watching a movie I get completely immersed in the alternative reality of the film,Movies change my position on social economic or political issues,When watching movies things get so intense that I have to stop watching,Gender identity (1 = female; 2 = male; 3 = self-described),Are you an only child? (1: Yes; 0: No; -1: Did not respond),Movies are best enjoyed alone (1: Yes; 0: No; -1: Did not respond)
0,NaN,NaN,4.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,1.0,6.0,2.0,5.0,5.0,5.0,1.0,1.0,0,1
1,NaN,NaN,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,1.0,1.0,6.0,5.0,3.0,2.0,1.0,0,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,4.0,3.0,5.0,5.0,4.0,4.0,1.0,1,0
3,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,4.0,...,3.0,1.0,1.0,4.0,5.0,3.0,1.0,1.0,0,1
4,NaN,NaN,3.5,NaN,0.5,NaN,0.5,1.0,NaN,0.0,...,2.0,3.0,2.0,5.0,6.0,4.0,4.0,1.0,1,1


In [69]:
df.shape

(1097, 477)

In [70]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
The Life of David Gale (2003),76.0,2.151316,1.166247,0.0,1.0,2.5,3.0,4.0
Wing Commander (1999),71.0,2.021127,1.141542,0.0,1.0,2.0,3.0,4.0
Django Unchained (2012),453.0,3.153422,0.916403,0.0,3.0,3.5,4.0,4.0
Alien (1979),289.0,2.707612,1.153261,0.0,2.0,3.0,3.5,4.0
Indiana Jones and the Last Crusade (1989),463.0,2.778618,0.906993,0.0,2.5,3.0,3.5,4.0
...,...,...,...,...,...,...,...,...
Movies change my position on social economic or political issues,1077.0,3.319406,1.246603,1.0,3.0,3.0,4.0,6.0
When watching movies things get so intense that I have to stop watching,1079.0,2.159407,1.275058,1.0,1.0,2.0,3.0,6.0
Gender identity (1 = female; 2 = male; 3 = self-described),1073.0,1.253495,0.447890,1.0,1.0,1.0,1.0,3.0
Are you an only child? (1: Yes; 0: No; -1: Did not respond),1097.0,0.137648,0.407743,-1.0,0.0,0.0,0.0,1.0


In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1097 entries, 0 to 1096
Columns: 477 entries, The Life of David Gale (2003) to Movies are best enjoyed alone (1: Yes; 0: No; -1: Did not respond)
dtypes: float64(475), int64(2)
memory usage: 4.0 MB


In [72]:
df.isnull().sum()

The Life of David Gale (2003)                                              1021
Wing Commander (1999)                                                      1026
Django Unchained (2012)                                                     644
Alien (1979)                                                                808
Indiana Jones and the Last Crusade (1989)                                   634
                                                                           ... 
Movies change my position on social economic or political issues             20
When watching movies things get so intense that I have to stop watching      18
Gender identity (1 = female; 2 = male; 3 = self-described)                   24
Are you an only child? (1: Yes; 0: No; -1: Did not respond)                   0
Movies are best enjoyed alone (1: Yes; 0: No; -1: Did not respond)            0
Length: 477, dtype: int64

In [73]:
df.columns

Index(['The Life of David Gale (2003)', 'Wing Commander (1999)',
       'Django Unchained (2012)', 'Alien (1979)',
       'Indiana Jones and the Last Crusade (1989)', 'Snatch (2000)',
       'Rambo: First Blood Part II (1985)', 'Fargo (1996)',
       'Let the Right One In (2008)', 'Black Swan (2010)',
       ...
       'When watching a movie I cheer or shout or talk or curse at the screen',
       'When watching a movie I feel like the things on the screen are happening to me',
       'As a movie unfolds I start to have problems keeping track of events that happened earlier',
       'The emotions on the screen "rub off" on me - for instance if something sad is happening I get sad or if something frightening is happening I get scared',
       'When watching a movie I get completely immersed in the alternative reality of the film',
       'Movies change my position on social economic or political issues',
       'When watching movies things get so intense that I have to stop watching',
 

In [74]:
df.head(20)

,The Life of David Gale (2003),Wing Commander (1999),Django Unchained (2012),Alien (1979),Indiana Jones and the Last Crusade (1989),Snatch (2000),Rambo: First Blood Part II (1985),Fargo (1996),Let the Right One In (2008),Black Swan (2010),...,When watching a movie I cheer or shout or talk or curse at the screen,When watching a movie I feel like the things on the screen are happening to me,As a movie unfolds I start to have problems keeping track of events that happened earlier,"The emotions on the screen ""rub off"" on me - for instance if something sad is happening I get sad or if something frightening is happening I get scared",When watching a movie I get completely immersed in the alternative reality of the film,Movies change my position on social economic or political issues,When watching movies things get so intense that I have to stop watching,Gender identity (1 = female; 2 = male; 3 = self-described),Are you an only child? (1: Yes; 0: No; -1: Did not respond),Movies are best enjoyed alone (1: Yes; 0: No; -1: Did not respond)
0,NaN,NaN,4.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,1.0,6.0,2.0,5.0,5.0,5.0,1.0,1.0,0,1
1,NaN,NaN,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,1.0,1.0,6.0,5.0,3.0,2.0,1.0,0,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,4.0,3.0,5.0,5.0,4.0,4.0,1.0,1,0
3,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,4.0,...,3.0,1.0,1.0,4.0,5.0,3.0,1.0,1.0,0,1
4,NaN,NaN,3.5,NaN,0.5,NaN,0.5,1.0,NaN,0.0,...,2.0,3.0,2.0,5.0,6.0,4.0,4.0,1.0,1,1
5,3.0,NaN,NaN,2.5,2.0,NaN,NaN,NaN,NaN,4.0,...,4.0,4.0,1.0,4.0,5.0,3.0,1.0,1.0,1,0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,1.0,5.0,3.0,5.0,5.0,4.0,3.0,1.0,0,0
7,3.5,NaN,3.5,2.5,3.0,3.0,3.0,3.5,4.0,4.0,...,4.0,5.0,3.0,5.0,5.0,4.0,3.0,1.0,0,1
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,5.0,3.0,6.0,5.0,1.0,3.0,1.0,0,1
9,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,3.0,...,1.0,4.0,3.0,5.0,5.0,3.0,2.0,2.0,-1,1


In [79]:
linerreg=LinearRegression()
imp=IterativeImputer(estimator=linerreg,missing_values=np.nan,max_iter=5,verbose=2,imputation_order='roman',random_state=0)
ImputedData=imp.fit_transform(df)

[IterativeImputer] Completing matrix with shape (1097, 477)
[IterativeImputer] Ending imputation round 1/5, elapsed time 14.64
[IterativeImputer] Change: 271.5245536746528, scaled tolerance: 0.006 
[IterativeImputer] Ending imputation round 2/5, elapsed time 26.98
[IterativeImputer] Change: 101.74394733987486, scaled tolerance: 0.006 
[IterativeImputer] Ending imputation round 3/5, elapsed time 39.73
[IterativeImputer] Change: 51.92409250390409, scaled tolerance: 0.006 
[IterativeImputer] Ending imputation round 4/5, elapsed time 52.82
[IterativeImputer] Change: 31.91376014432724, scaled tolerance: 0.006 
[IterativeImputer] Ending imputation round 5/5, elapsed time 65.89
[IterativeImputer] Change: 23.12598077242755, scaled tolerance: 0.006 


C:\Users\PAJJU\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [83]:
pd.DataFrame(ImputedData)

,0,1,2,3,4,5,6,7,8,9,...,467,468,469,470,471,472,473,474,475,476
0,1.864024,1.535157,4.000000,1.538175,3.000000,3.438163,0.640139,3.882275,3.175945,2.911565,...,1.0,6.0,2.0,5.0,5.0,5.0,1.0,1.0,0.0,1.0
1,2.793854,2.681752,1.500000,1.628505,2.023463,1.816171,2.704390,2.875461,2.466458,2.911565,...,3.0,1.0,1.0,6.0,5.0,3.0,2.0,1.0,0.0,0.0
2,3.047908,2.708265,4.271560,2.155238,2.893093,2.122207,2.285263,3.681163,2.896771,2.911565,...,5.0,4.0,3.0,5.0,5.0,4.0,4.0,1.0,1.0,0.0
3,2.087984,2.189336,2.000000,3.964571,3.000000,3.926463,2.120165,3.008778,4.459961,4.000000,...,3.0,1.0,1.0,4.0,5.0,3.0,1.0,1.0,0.0,1.0
4,2.792585,3.218181,3.500000,0.795445,0.500000,2.303072,0.500000,1.000000,0.685727,0.000000,...,2.0,3.0,2.0,5.0,6.0,4.0,4.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,2.977788,2.635405,3.736736,2.913589,3.500000,1.998499,2.725637,3.344269,3.149107,2.911565,...,3.0,4.0,3.0,5.0,5.0,4.0,4.0,1.0,0.0,0.0
1093,3.000000,4.000000,8.440361,3.932997,4.000000,4.000000,2.500000,4.160160,3.500000,3.500000,...,5.0,3.0,5.0,5.0,5.0,6.0,5.0,1.0,0.0,0.0
1094,2.735052,2.989036,4.282638,3.201651,3.150560,3.335663,3.803859,3.500000,4.068417,2.911565,...,6.0,3.0,1.0,6.0,6.0,4.0,2.0,1.0,0.0,0.0
1095,2.713727,2.661375,1.519174,3.227390,3.038260,2.380206,0.236314,1.806770,2.676871,2.911565,...,1.0,1.0,1.0,4.0,3.0,3.0,1.0,1.0,0.0,1.0
